In [0]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/nltk_project/')

In [0]:
from zipfile import ZipFile
with ZipFile('blog-authorship-corpus.zip', 'r') as z:
  z.extractall()


In [7]:
ls


blog-authorship-corpus.zip  blogtext.csv


In [6]:
import nltk
nltk.download()   # Run this the first time after installing nltk package. It will open a GUI. Select all and hit Download.


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [0]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [0]:
# 1.	Load the dataset (5 points) 
  # a.	Tip: As the dataset is large, use fewer rows. Check what is working well on your machine and decide accordingly
blog_text = pd.read_csv('blogtext.csv')

In [0]:
# Taking first 50000 rows
blog_text = blog_text.head(1000)



In [11]:
blog_text.shape

(50000, 7)

In [0]:
# 2. Preprocess rows of the “text” column (7.5 points)
  # a.	Remove unwanted characters 
  # b.	Convert text to lowercase 
  # c.	Remove unwanted spaces 
  # d.	Remove stopwords
from nltk.corpus import stopwords
blog_text['text'] = blog_text['text'].str.lower()

In [0]:
blog_text['text'] = blog_text['text'].apply(lambda x: remove_spaces(x))

In [0]:
blog_text['text'] = blog_text['text'].apply(lambda x: remove_punctuation(x))

In [0]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

blog_text['text'] = blog_text['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [0]:
blog_text['text'] = blog_text['text'].apply(lambda x: remove_non_english_words(x))

In [0]:
import string 
words = set(nltk.corpus.words.words())

def remove_spaces(text):
    return " ".join(text.split())

def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator) 

def remove_non_english_words(text):

  return " ".join(w for w in nltk.wordpunct_tokenize(text) \

         if w.lower() in words )


In [18]:
blog_text['text'][5353]

'hour long conversation hair guy never met life hair blue kept thinking another one psychotic ate chocolate ever thought possible ha ha record crazy oh wow deep breath laugh sigh'

In [0]:
# blog_text['text1'][5353]

In [159]:
# 3.	As we want to make this into a multi-label classification problem, you are required to merge all the label columns together, so that we have all the labels together for a particular sentence (7.5 points) 
  # a.	Label columns to merge: “gender”, “age”, “topic”, “sign” 
  # b.	After completing the previous step, there should be only two columns in your data frame i.e. “text” and “labels” as shown in the below image

blog_text.head()

  

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004",found wait untill team leader
1,2059027,male,15,Student,Leo,"13,May,2004",team van mail mail mail
2,2059027,male,15,Student,Leo,"12,May,2004",het van build subject build humorous date orga...
3,2059027,male,15,Student,Leo,"12,May,2004",testing testing
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks capture show cool links pop audio video...


In [0]:
blog_text['labels'] = blog_text[['gender','age','topic','sign']].apply(lambda x: creating_label(x), axis=1)

In [0]:
def creating_label(text): 
    column_list = []
    column_list.append(text['gender'])
    column_list.append(text['age'])
    column_list.append(text['topic'])
    column_list.append(text['sign'])
    return column_list

In [162]:
blog_text.head()

,id,gender,age,topic,sign,date,text,labels
0,2059027,male,15,Student,Leo,"14,May,2004",found wait untill team leader,"[male, 15, Student, Leo]"
1,2059027,male,15,Student,Leo,"13,May,2004",team van mail mail mail,"[male, 15, Student, Leo]"
2,2059027,male,15,Student,Leo,"12,May,2004",het van build subject build humorous date orga...,"[male, 15, Student, Leo]"
3,2059027,male,15,Student,Leo,"12,May,2004",testing testing,"[male, 15, Student, Leo]"
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks capture show cool links pop audio video...,"[male, 33, InvestmentBanking, Aquarius]"


In [0]:
blog_text = blog_text.drop(['gender','age','topic','sign','id','date'], axis = 1)


In [164]:

blog_text.head()


,text,labels
0,found wait untill team leader,"[male, 15, Student, Leo]"
1,team van mail mail mail,"[male, 15, Student, Leo]"
2,het van build subject build humorous date orga...,"[male, 15, Student, Leo]"
3,testing testing,"[male, 15, Student, Leo]"
4,thanks capture show cool links pop audio video...,"[male, 33, InvestmentBanking, Aquarius]"


In [0]:
# 4. Separate features and labels, and split the data into training and testing (5 points)

# define X and y
X = blog_text['text']
y = blog_text['labels']

# split the new DataFrame into training and testing sets [Default test size = 25%]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [192]:
print(X_train.shape)
print(y_test.shape)

(750,)
(250,)


In [0]:
# 5.	Vectorize the features (5 points) 
#   a.	Create a Bag of Words using count vectorizer 
      # i.	Use ngram_range=(1, 2) 
      # ii.	Vectorize training and testing features 
  # b.	Print the term-document matrix

vect = CountVectorizer(ngram_range=(1, 2))
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [194]:
X_train_dtm.shape

(750, 72476)

In [195]:
# last 50 features
print(vect.get_feature_names()[-50:])

['yr happening', 'yr hovering', 'yr last', 'yr new', 'yr oh', 'yr old', 'yr principal', 'yr pro', 'yr projection', 'yr really', 'yr since', 'yr theory', 'yr thing', 'yr toiletry', 'yr used', 'yr went', 'yr would', 'yuan', 'yuan day', 'yucca', 'yucca improve', 'yuck', 'yuck anyway', 'yummy', 'yummy colors', 'yummy food', 'yummy recipe', 'zero', 'zero help', 'zero quite', 'zest', 'zest close', 'zimbabwe', 'zimbabwe republic', 'zipping', 'zipping past', 'zodiac', 'zodiac sign', 'zodiac similar', 'zombie', 'zombie right', 'zone', 'zone beyond', 'zone hope', 'zone large', 'zone one', 'zone went', 'zone work', 'zoo', 'zoo face']


In [0]:
# df = pd.DataFrame(X_train_dtm.head(10).toarray(), columns=vect.get_feature_names()[10:])
# print(df)

In [0]:
# 6.	Create a dictionary to get the count of every label i.e. the key will be label name and value will be the total count of the label. Check below image for reference (5 points)


df = pd.Series(sum([item for item in y_train], [])).value_counts()


In [220]:
df.to_dict() 


{14: 50,
 15: 51,
 17: 108,
 23: 55,
 24: 248,
 25: 45,
 26: 28,
 27: 62,
 33: 61,
 34: 6,
 37: 12,
 41: 11,
 44: 1,
 45: 12,
 'Aquarius': 161,
 'Aries': 33,
 'Arts': 1,
 'Banking': 8,
 'BusinessServices': 16,
 'Cancer': 56,
 'Capricorn': 64,
 'Communications-Media': 11,
 'Education': 62,
 'Engineering': 84,
 'Gemini': 11,
 'InvestmentBanking': 53,
 'Leo': 30,
 'Libra': 127,
 'Non-Profit': 37,
 'Pisces': 1,
 'Sagittarius': 75,
 'Science': 19,
 'Scorpio': 186,
 'Sports-Recreation': 62,
 'Student': 109,
 'Taurus': 4,
 'Virgo': 2,
 'female': 320,
 'indUnk': 288,
 'male': 430}

In [0]:
# 7.Transform the labels - (7.5 points) As we have noticed before, in this task each example can have multiple tags. To deal with such kind of prediction, we need to transform labels in a binary form and the prediction will be a mask of 0s and 1s. For this purpose, it is convenient to use MultiLabelBinarizer from sklearn 
#   a.	Convert your train and test labels using MultiLabelBinarizer

j = 0
for x in y_test:
  ind = y_test.index[j]
  y_test[ind] = [str(i) for i in x]
  j = j+1

In [0]:
j = 0
for x in y_train:
  ind = y_train.index[j]
  y_train[ind] = [str(i) for i in x]
  j = j+1

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb_train = MultiLabelBinarizer()
mlb_train_data = mlb_train.fit_transform(y_train.to_list())

In [0]:

mlb_test = mlb_train.transform(y_test.to_list())

In [226]:
mlb_train.classes_

array(['14', '15', '17', '23', '24', '25', '26', '27', '33', '34', '37',
       '41', '44', '45', 'Aquarius', 'Aries', 'Arts', 'Banking',
       'BusinessServices', 'Cancer', 'Capricorn', 'Communications-Media',
       'Education', 'Engineering', 'Gemini', 'InvestmentBanking', 'Leo',
       'Libra', 'Non-Profit', 'Pisces', 'Sagittarius', 'Science',
       'Scorpio', 'Sports-Recreation', 'Student', 'Taurus', 'Virgo',
       'female', 'indUnk', 'male'], dtype=object)

In [227]:
mlb_test.shape

(250, 40)

In [205]:
y_train.shape

(750,)

In [228]:
mlb_train.classes_

array(['14', '15', '17', '23', '24', '25', '26', '27', '33', '34', '37',
       '41', '44', '45', 'Aquarius', 'Aries', 'Arts', 'Banking',
       'BusinessServices', 'Cancer', 'Capricorn', 'Communications-Media',
       'Education', 'Engineering', 'Gemini', 'InvestmentBanking', 'Leo',
       'Libra', 'Non-Profit', 'Pisces', 'Sagittarius', 'Science',
       'Scorpio', 'Sports-Recreation', 'Student', 'Taurus', 'Virgo',
       'female', 'indUnk', 'male'], dtype=object)

In [0]:

# 8.	 Choose a classifier - (5 points) In this task, we suggest using the One-vs-Rest approach, which is implemented in OneVsRestClassifier class. In this approach k classifiers (= number of tags) are trained. As a basic classifier, use LogisticRegression. It is one of the simplest methods, but often it performs good enough in text classification tasks. It might take some time because the number of classifiers to train is large. 
#       a.	Use a linear classifier of your choice, wrap it up in OneVsRestClassifier to train it on every label 
#       b.	As One-vs-Rest approach might not have been discussed in the sessions, we are providing you the code for that

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver = 'lbfgs')
clf = OneVsRestClassifier(clf)

In [230]:
# 9.	Fit the classifier, make predictions and get the accuracy (5 points) 
#     a.	Print the following 
#       i.	Accuracy score 
#       ii.	F1 score 
#       iii.	Average precision score 
#       iv.	Average recall score v.	Tip: Make sure you are familiar with all of them. How would you expect the things to work for the multi-label scenario? Read about micro/macro/weighted averaging


clf.fit(X_train_dtm,mlb_train_data)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [231]:
X_test_dtm.shape

(250, 72476)

In [0]:
y_pred = clf.predict(X_test_dtm)

In [233]:
y_pred.shape

(250, 40)

In [234]:
print(metrics.accuracy_score(mlb_test, y_pred))


0.172


In [235]:
print(metrics.f1_score(mlb_test, y_pred, average='macro'))

0.27173211695640764


In [236]:
print(metrics.precision_score(mlb_test, y_pred, average='macro'))

0.5206997313873186


In [237]:
print(metrics.recall_score(mlb_test, y_pred, average='macro'))

0.2037730228308003
